In [43]:

from functools import partial
from pathlib import Path
import numpy as np
import lnoi400
import gdsfactory as gf
import os
import sys
# sys.path.insert(0, "C:/Users/Steffen/Documents/GitHub/MWLab-Library/scripts")
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), "../../scripts/")))
import devices
import components

gf.clear_cache()

def to_itype(points, dbu):
    return [(int(round(x / dbu)), int(round(y / dbu))) for x, y in points]
dbu = 0.001

@gf.cell
def chip_frame():
    c = gf.get_component("chip_frame", size=(10_000, 5000), center=(5050, 2525))
    return c

chip_layout = chip_frame()

input_ext = 10.0
double_taper = gf.get_component("double_linear_inverse_taper",input_ext=input_ext,)

routing_roc = 50.0
ports_gap = 50
frame = 50

trav_wave_eom = devices.tWave_EOM()

n = 0

device_xoffset = 180.0 + frame
device_yoffset = 160.0 + frame
in_port_xoffset = 100
out_port_xoffset = -70 

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def trave_wave_eom_circuit(
    trav_wave_eom = trav_wave_eom,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    trav_wave_eom_ref = c << trav_wave_eom
    trav_wave_eom_ref.dmovex(- trav_wave_eom_ref.ports["o1"].dcenter[0] + device_xoffset)
    trav_wave_eom_ref.dmovey(chip_layout.dymax - trav_wave_eom_ref.dymax - device_yoffset)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        trav_wave_eom_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        trav_wave_eom_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        trav_wave_eom_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

circuit = trave_wave_eom_circuit()

######################################

######################################

routing_xoffset = 0.04 # percentage of the horizontal size of a device
routing_yoffset = 0.50

eom_ec_emmi = devices.dOR_EOM_DC_EMMI()

n = 1

device_xoffset = 180.0 + frame
device_yoffset = 130.0 + frame + eom_ec_emmi.dysize
in_port_xoffset = 100 + n * ports_gap
out_port_xoffset = -70 + ports_gap

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def eom_ec_emmi_circuit2(
    eom_ec_emmi = eom_ec_emmi,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    eom_ec_emmi_ref = c << eom_ec_emmi
    eom_ec_emmi_ref.dmovex(- eom_ec_emmi_ref.ports["o1"].dcenter[0] + 2 * device_xoffset)
    eom_ec_emmi_ref.dmovey(chip_layout.dymax - eom_ec_emmi_ref.dysize - 0.6 * device_yoffset)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o2"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 5),
                   (ecs["out1"].ports["o2"].dcenter[0], eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 5)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize + 5, eom_ec_emmi_ref.ports["o3"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize + 5, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 3.5),
                   (ecs["out2"].ports["o2"].dcenter[0], eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 3.5)]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

ni = 2
no = 2
device_xoffset = 180.0 + frame
device_yoffset = 130.0 + frame + eom_ec_emmi.dysize
in_port_xoffset = 100 + ni * ports_gap
out_port_xoffset = -70 + no * ports_gap

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def eom_ec_emmi_circuit3(
    eom_ec_emmi = eom_ec_emmi,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    eom_ec_emmi_ref = c << eom_ec_emmi
    eom_ec_emmi_ref.dmovex(- eom_ec_emmi_ref.ports["o1"].dcenter[0] + eom_ec_emmi_ref.dxsize + 3 * device_xoffset)
    eom_ec_emmi_ref.dmovey(chip_layout.dymax - eom_ec_emmi_ref.dysize - 0.6 * device_yoffset)

    global circuit3_saved_out_xwaypoint
    circuit3_saved_out_xwaypoint = eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize + 5
    global circuit3_saved_out_ywaypoint
    circuit3_saved_out_ywaypoint = eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 6.5

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o1"].dcenter[0] - routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o1"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o1"].dcenter[0]- routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o1"].dcenter[1] - routing_yoffset * eom_ec_emmi_ref.dysize),
                   (ecs["in"].ports["o2"].dcenter[0], eom_ec_emmi_ref.ports["o1"].dcenter[1] - routing_yoffset * eom_ec_emmi_ref.dysize)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o2"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 5),
                   (ecs["out1"].ports["o2"].dcenter[0], eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 5)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize + 5, eom_ec_emmi_ref.ports["o3"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize + 5, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 6.5),
                   (ecs["out2"].ports["o2"].dcenter[0], eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 6.5)]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

circuit3 = eom_ec_emmi_circuit3()

ni = 3
no = 3
device_xoffset = 180.0 + frame
device_yoffset = 130.0 + frame + eom_ec_emmi.dysize
in_port_xoffset = 100 + ni * ports_gap
out_port_xoffset = -70 + no *  ports_gap

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)


ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def eom_ec_emmi_circuit4(
    eom_ec_emmi = eom_ec_emmi,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    eom_ec_emmi_ref = c << eom_ec_emmi
    eom_ec_emmi_ref.dmovex(- eom_ec_emmi_ref.ports["o1"].dcenter[0] + 2 * device_xoffset)
    eom_ec_emmi_ref.dmovey(chip_layout.dymax - eom_ec_emmi_ref.dysize - 1.4 * device_yoffset)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o2"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 5),
                   (circuit3_saved_out_xwaypoint + 5, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 5),
                   (circuit3_saved_out_xwaypoint + 5, circuit3_saved_out_ywaypoint - 5),
                   (ecs["out1"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 5)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize + 6, eom_ec_emmi_ref.ports["o3"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset * eom_ec_emmi_ref.dxsize + 6, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 2.5),
                   (circuit3_saved_out_xwaypoint + 10, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 2.5),
                   (circuit3_saved_out_xwaypoint + 10, circuit3_saved_out_ywaypoint - 10),
                   (ecs["out2"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 10)]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

ni = 4
no = 4
device_xoffset = 180.0 + frame
device_yoffset = 130.0 + frame + eom_ec_emmi.dysize
in_port_xoffset = 100 + ni * ports_gap
out_port_xoffset = -70 + no * ports_gap

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def eom_ec_emmi_circuit5(
    eom_ec_emmi = eom_ec_emmi,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    eom_ec_emmi_ref = c << eom_ec_emmi
    eom_ec_emmi_ref.dmovex(- eom_ec_emmi_ref.ports["o1"].dcenter[0] + eom_ec_emmi_ref.dxsize + 3 * device_xoffset)
    eom_ec_emmi_ref.dmovey(chip_layout.dymax - eom_ec_emmi_ref.dysize - 1.4 * device_yoffset)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o1"].dcenter[0] - routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o1"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o1"].dcenter[0]- routing_xoffset * eom_ec_emmi_ref.dxsize, eom_ec_emmi_ref.ports["o1"].dcenter[1] - routing_yoffset * eom_ec_emmi_ref.dysize),
                   (ecs["in"].ports["o2"].dcenter[0], eom_ec_emmi_ref.ports["o1"].dcenter[1] - routing_yoffset * eom_ec_emmi_ref.dysize)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(circuit3_saved_out_xwaypoint + 15, eom_ec_emmi_ref.ports["o2"].dcenter[1]),
                   (circuit3_saved_out_xwaypoint + 15, circuit3_saved_out_ywaypoint - 15),
                   (ecs["out1"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 15)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(circuit3_saved_out_xwaypoint + 20, eom_ec_emmi_ref.ports["o3"].dcenter[1]),
                   (circuit3_saved_out_xwaypoint + 20, circuit3_saved_out_ywaypoint - 20),
                   (ecs["out2"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 20)]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c



####################################

####################################

routing_xoffset_eom_ec = 200 
routing_yoffset = 0.50

eom_ec_emmi = devices.dOR_EOM_DC_EMMI(
    lextra = 205/2,
    ls = 3850
)

n = 5

device_xoffset = 180.0 + frame
device_yoffset = 130.0 + frame + eom_ec_emmi.dysize
in_port_xoffset = 100 + n * ports_gap
out_port_xoffset = -70 + n * ports_gap

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def eom_ec_emmi_circuit6(
    eom_ec_emmi = eom_ec_emmi,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    eom_ec_emmi_ref = c << eom_ec_emmi
    eom_ec_emmi_ref.dmovex(- eom_ec_emmi_ref.ports["o1"].dcenter[0] + 2.1 * device_xoffset)
    eom_ec_emmi_ref.dmovey(chip_layout.dymax - 3 * eom_ec_emmi_ref.dysize - 0.7 * device_yoffset)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset_eom_ec, eom_ec_emmi_ref.ports["o2"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset_eom_ec, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 5),
                   (circuit3_saved_out_xwaypoint + 25, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 5),
                   (circuit3_saved_out_xwaypoint + 25, circuit3_saved_out_ywaypoint - 25),
                   (ecs["out1"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 25)]
                   
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 5, eom_ec_emmi_ref.ports["o3"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 5, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 2.5),
                   (circuit3_saved_out_xwaypoint + 30, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize + 2.5),
                   (circuit3_saved_out_xwaypoint + 30, circuit3_saved_out_ywaypoint - 30),
                   (ecs["out2"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 30)]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

eom_ec_emmi = devices.dOR_EOM_DC_EMMI(
    lextra = 205/2,
    ls = 3850
)

n = 6

device_xoffset = 180.0 + frame
device_yoffset = 500.0 + frame + eom_ec_emmi.dysize
in_port_xoffset = 100 + n * ports_gap
out_port_xoffset = -70 + n * ports_gap

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def eom_ec_emmi_circuit7(
    eom_ec_emmi = eom_ec_emmi,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    eom_ec_emmi_ref = c << eom_ec_emmi
    eom_ec_emmi_ref.dmovex(- eom_ec_emmi_ref.ports["o1"].dcenter[0] + 2.1 * device_xoffset + ports_gap)
    eom_ec_emmi_ref.dmovey(chip_layout.dymax - 4 * eom_ec_emmi_ref.dysize - 0.8 * device_yoffset)

    global circuit7_saved_out_waypoint, circuit7_saved_out_waypoint_list
    circuit7_saved_out_waypoint = eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 15 - ports_gap
    circuit7_saved_out_waypoint_list = [(eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 15 - ports_gap, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 9.5 + 1 * eom_ec_emmi_ref.dysize + 82.42),
                   (circuit3_saved_out_xwaypoint + 40, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 9.5 + 1 * eom_ec_emmi_ref.dysize + 82.42),
                   (circuit3_saved_out_xwaypoint + 40, circuit3_saved_out_ywaypoint - 40),
                   (ecs["out2"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 40)]

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset_eom_ec + 10 - ports_gap, eom_ec_emmi_ref.ports["o2"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset_eom_ec + 10 - ports_gap, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 7 + 1 * eom_ec_emmi_ref.dysize + 82.42),
                   (circuit3_saved_out_xwaypoint + 35, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 7 + 1 * eom_ec_emmi_ref.dysize + 82.42),
                   (circuit3_saved_out_xwaypoint + 35, circuit3_saved_out_ywaypoint - 35),
                   (ecs["out1"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 35)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 15 - ports_gap, eom_ec_emmi_ref.ports["o3"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 15 - ports_gap, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 9.5 + 1 * eom_ec_emmi_ref.dysize + 82.42),
                   (circuit3_saved_out_xwaypoint + 40, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 9.5 + 1 * eom_ec_emmi_ref.dysize + 82.42),
                   (circuit3_saved_out_xwaypoint + 40, circuit3_saved_out_ywaypoint - 40),
                   (ecs["out2"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 40)] 
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

#### 
# Run with this line uncommented once and then run again with it commented
""" circuit7 = eom_ec_emmi_circuit7() """
#circuit7 = eom_ec_emmi_circuit7()

eom_ec_emmi = devices.dOR_EOM_DC_EMMI(
    lextra = 205/2,
    ls = 3850
)

n = 7

device_xoffset = 180.0 + frame
device_yoffset = 130.0 + frame + eom_ec_emmi.dysize
in_port_xoffset = 100 + n * ports_gap
out_port_xoffset = -70 + n * ports_gap

x_in = frame + in_port_xoffset
in_loc = np.array([x_in, 0])
out_loc = np.array([x_in + device_xoffset + trav_wave_eom.dxsize + out_port_xoffset, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def eom_ec_emmi_circuit8(
    eom_ec_emmi = eom_ec_emmi,
    ecs = ecs,
) -> gf.Component:

    c = gf.Component()
    eom_ec_emmi_ref = c << eom_ec_emmi
    eom_ec_emmi_ref.dmovex(- eom_ec_emmi_ref.ports["o1"].dcenter[0] + 2.1 * device_xoffset + 2 * ports_gap)
    eom_ec_emmi_ref.dmovey(chip_layout.dymax - 5 * eom_ec_emmi_ref.dysize - 0.7 * device_yoffset)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset_eom_ec + 15 - 2 * ports_gap, eom_ec_emmi_ref.ports["o2"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o2"].dcenter[0] + routing_xoffset_eom_ec + 15 - 2 * ports_gap, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 19 + 2 * eom_ec_emmi_ref.dysize),
                   (circuit3_saved_out_xwaypoint + 45, eom_ec_emmi_ref.ports["o2"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 19 + 2 * eom_ec_emmi_ref.dysize),
                   (circuit3_saved_out_xwaypoint + 45, circuit3_saved_out_ywaypoint - 45),
                   (ecs["out1"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 45)]
    )

    gf.routing.route_single(
        c,
        eom_ec_emmi_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 20 - 2 * ports_gap, eom_ec_emmi_ref.ports["o3"].dcenter[1]),
                   (eom_ec_emmi_ref.ports["o3"].dcenter[0] + routing_xoffset_eom_ec + 20 - 2 * ports_gap, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 21.5 + 2 * eom_ec_emmi_ref.dysize),
                   (circuit3_saved_out_xwaypoint + 50, eom_ec_emmi_ref.ports["o3"].dcenter[1] + routing_yoffset * eom_ec_emmi_ref.dysize - 21.5 + 2 * eom_ec_emmi_ref.dysize),
                   (circuit3_saved_out_xwaypoint + 50, circuit3_saved_out_ywaypoint - 50),
                   (ecs["out2"].ports["o2"].dcenter[0], circuit3_saved_out_ywaypoint - 50)]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

duel_eo_comb = devices.dualEOcomb_sbend()

n = 1

device_gap = duel_eo_comb.dysize + 250

device_xoffset2 = 150 + frame
device_yoffset2 = -100.0 + frame + duel_eo_comb.dysize

routing_xoffset2 = 150
routing_yoffset2 = 600

in_port_xoffset = chip_layout.dxmax - duel_eo_comb.dxsize - device_xoffset2 + frame - 80
out_port_xoffset = chip_layout.dxmax - 1200 

x_in =  in_port_xoffset - frame - (6 - n) * ports_gap
x_out = out_port_xoffset - frame + (2 * n) * ports_gap
in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def duel_eo_comb_circuit9(
    duel_eo_comb = duel_eo_comb,
    ecs = ecs,
    n = 1,
) -> gf.Component:

    c = gf.Component()
    duel_eo_comb_ref = c << duel_eo_comb
    duel_eo_comb_ref.dmovex(chip_layout.dxmax - duel_eo_comb_ref.ports["o1"].dcenter[0] - duel_eo_comb_ref.dxsize - device_xoffset2 )
    duel_eo_comb_ref.dmovey(chip_layout.dymax - duel_eo_comb_ref.ports["o1"].dcenter[1] - duel_eo_comb_ref.dysize - device_yoffset2 - (n - 1) * device_gap)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, duel_eo_comb_ref.ports["o2"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5),
                   (ecs["out1"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5)]
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), duel_eo_comb_ref.ports["o3"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4)),
                   (ecs["out2"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4))]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

duel_eo_comb = devices.dualEOcomb_sbend()

n = 2

x_in =  in_port_xoffset - frame - (6 - n) * ports_gap
x_out = out_port_xoffset - frame + (2 * n) * ports_gap
in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def duel_eo_comb_circuit10(
    duel_eo_comb = duel_eo_comb,
    ecs = ecs,
    n = 2,
) -> gf.Component:

    c = gf.Component()
    duel_eo_comb_ref = c << duel_eo_comb
    duel_eo_comb_ref.dmovex(chip_layout.dxmax - duel_eo_comb_ref.ports["o1"].dcenter[0] - duel_eo_comb_ref.dxsize - device_xoffset2)
    duel_eo_comb_ref.dmovey(chip_layout.dymax - duel_eo_comb_ref.ports["o1"].dcenter[1] - duel_eo_comb_ref.dysize - 1 * device_yoffset2 - (n - 1) * device_gap)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, duel_eo_comb_ref.ports["o2"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5),
                   (ecs["out1"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5)]
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), duel_eo_comb_ref.ports["o3"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4)),
                   (ecs["out2"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4))]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

duel_eo_comb = devices.dualEOcomb_sbend()

n = 3

x_in =  in_port_xoffset - frame - (6 - n) * ports_gap
x_out = out_port_xoffset - frame + (2 * n) * ports_gap
in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def duel_eo_comb_circuit11(
    duel_eo_comb = duel_eo_comb,
    ecs = ecs,
    n = 3,
) -> gf.Component:

    c = gf.Component()
    duel_eo_comb_ref = c << duel_eo_comb
    duel_eo_comb_ref.dmovex(chip_layout.dxmax - duel_eo_comb_ref.ports["o1"].dcenter[0] - duel_eo_comb_ref.dxsize - device_xoffset2)
    duel_eo_comb_ref.dmovey(chip_layout.dymax - duel_eo_comb_ref.ports["o1"].dcenter[1] - duel_eo_comb_ref.dysize - 1 * device_yoffset2 - (n - 1) * device_gap)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, duel_eo_comb_ref.ports["o2"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5),
                   (ecs["out1"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5)]
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), duel_eo_comb_ref.ports["o3"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4)),
                   (ecs["out2"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4))]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

duel_eo_comb = devices.dualEOcomb_sbend()

n = 4

x_in =  in_port_xoffset - frame - (6 - n) * ports_gap
x_out = out_port_xoffset - frame + (2 * n) * ports_gap
in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def duel_eo_comb_circuit12(
    duel_eo_comb = duel_eo_comb,
    ecs = ecs,
    n = 4,
) -> gf.Component:

    c = gf.Component()
    duel_eo_comb_ref = c << duel_eo_comb
    duel_eo_comb_ref.dmovex(chip_layout.dxmax - duel_eo_comb_ref.ports["o1"].dcenter[0] - duel_eo_comb_ref.dxsize - device_xoffset2)
    duel_eo_comb_ref.dmovey(chip_layout.dymax - duel_eo_comb_ref.ports["o1"].dcenter[1] - duel_eo_comb_ref.dysize - 1 * device_yoffset2 - (n - 1) * device_gap)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, duel_eo_comb_ref.ports["o2"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5),
                   (ecs["out1"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5)]
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), duel_eo_comb_ref.ports["o3"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4)),
                   (ecs["out2"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4))]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

duel_eo_comb = devices.dualEOcomb_sbend()

n = 5

x_in =  in_port_xoffset - frame - (6 - n) * ports_gap
x_out = out_port_xoffset - frame + (2 * n) * ports_gap
in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def duel_eo_comb_circuit13(
    duel_eo_comb = duel_eo_comb,
    ecs = ecs,
    n = 5,
) -> gf.Component:

    c = gf.Component()
    duel_eo_comb_ref = c << duel_eo_comb
    duel_eo_comb_ref.dmovex(chip_layout.dxmax - duel_eo_comb_ref.ports["o1"].dcenter[0] - duel_eo_comb_ref.dxsize - device_xoffset2)
    duel_eo_comb_ref.dmovey(chip_layout.dymax - duel_eo_comb_ref.ports["o1"].dcenter[1] - duel_eo_comb_ref.dysize - 1 * device_yoffset2 - (n - 1) * device_gap)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, duel_eo_comb_ref.ports["o2"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5),
                   (ecs["out1"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5)]
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), duel_eo_comb_ref.ports["o3"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4)),
                   (ecs["out2"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4))]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c

duel_eo_comb = devices.dualEOcomb_sbend()

n = 6

x_in =  in_port_xoffset - frame - (6 - n) * ports_gap
x_out = out_port_xoffset - frame + (2 * n) * ports_gap
in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [ports_gap, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ec_out2 = gf.Component()
ec_ref = ec_out2 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0, 0.5 * input_ext]
)
ec_out2.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1,
    "out2": ec_out2,
}

@gf.cell
def duel_eo_comb_circuit14(
    duel_eo_comb = duel_eo_comb,
    ecs = ecs,
    n = 6,
) -> gf.Component:

    c = gf.Component()
    duel_eo_comb_ref = c << duel_eo_comb
    duel_eo_comb_ref.dmovex(chip_layout.dxmax - duel_eo_comb_ref.ports["o1"].dcenter[0] - duel_eo_comb_ref.dxsize - device_xoffset2)
    duel_eo_comb_ref.dmovey(chip_layout.dymax - duel_eo_comb_ref.ports["o1"].dcenter[1] - duel_eo_comb_ref.dysize - 1 * device_yoffset2 - (n - 1) * device_gap)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, duel_eo_comb_ref.ports["o2"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6) + 5, chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5),
                   (ecs["out1"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4) + 5)]
    )

    gf.routing.route_single(
        c,
        duel_eo_comb_ref.ports["o3"],
        ecs["out2"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), duel_eo_comb_ref.ports["o3"].dcenter[1]),
                   (chip_layout.dxmax - routing_xoffset2 + 5 * (2 * n - 6), chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4)),
                   (ecs["out2"].ports["o2"].dcenter[0], chip_layout.dymax - routing_yoffset2 + 5 * (2 * n  - 4))]
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])
    c.add_port(name="o3", port=ecs["out2"].ports["o1"])

    return c


########################
# Single comb circuits #
########################

EOcomb = devices.EOcomb()

routing_xoffset_eo_comb = -680 
routing_yoffset_eo_comb = 250
routing_yoffset_eo_comb2 = 1600
device_gap_EOcomb = 50

device_xoffset_EOcomb = 770.0
device_yoffset_EOcomb = -300.0
in_port_xoffset_EOcomb = 700
out_port_xoffset_EOcomb = 1400 

n = 0

x_in = 100 + (n + 7) * ports_gap + frame
x_out = circuit7_saved_out_waypoint_list[-1][0] + (n + 1) * ports_gap

in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1
}

circuit15_routing_list = [(x + (n + 1) * 5, y - (n + 1) * 5) for (x, y) in circuit7_saved_out_waypoint_list]
circuit15_routing_list[-1] = (ecs["out1"].ports["o2"].dcenter[0], circuit15_routing_list[-1][1])

@gf.cell
def EOcomb_circuit15(
    EOcomb = EOcomb,
    ecs = ecs,
    n = 0
) -> gf.Component:

    c = gf.Component()
    EOcomb_ref = c << EOcomb
    EOcomb_ref.dmovex(-EOcomb_ref.ports["o1"].dcenter[0] + n * EOcomb_ref.dxsize + device_xoffset_EOcomb)
    EOcomb_ref.dmovey(-EOcomb_ref.ports["o1"].dcenter[1] + EOcomb_ref.dysize + device_yoffset_EOcomb + (2 - n) * 5)

    global circuit15_saved_out_waypoint
    circuit15_saved_out_waypoint = ecs["in"].ports["o2"].dcenter[0]

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000"
        
    )

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(EOcomb_ref.ports["o2"].dcenter[0] + EOcomb_ref.dxsize + routing_xoffset_eo_comb, EOcomb_ref.ports["o2"].dcenter[1]),
                   (EOcomb_ref.ports["o2"].dcenter[0] + EOcomb_ref.dxsize + routing_xoffset_eo_comb, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb + (n + 1) * 5),
                   (ecs["in"].ports["o2"].dcenter[0] + (n + 1) * 5, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb + (n + 1) * 5),
                   (ecs["in"].ports["o2"].dcenter[0] + (n + 1) * 5, routing_yoffset_eo_comb2 - (n + 1) * 5),
                   (circuit7_saved_out_waypoint + (n + 1) * 5, routing_yoffset_eo_comb2 - (n + 1) * 5)] + circuit15_routing_list
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])

    return c

n = 1

x_in = 100 + (n + 7) * ports_gap + frame
x_out = circuit7_saved_out_waypoint_list[-1][0] + (n + 1) * ports_gap

in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1
}

circuit16_routing_list = [(x + (n + 1) * 5, y - (n + 1) * 5) for (x, y) in circuit7_saved_out_waypoint_list]
circuit16_routing_list[-1] = (ecs["out1"].ports["o2"].dcenter[0], circuit16_routing_list[-1][1])

@gf.cell
def EOcomb_circuit16(
    EOcomb = EOcomb,
    ecs = ecs,
    n = 1
) -> gf.Component:

    c = gf.Component()
    EOcomb_ref = c << EOcomb
    EOcomb_ref.dmovex(-EOcomb_ref.ports["o1"].dcenter[0] + n * EOcomb_ref.dxsize + device_xoffset_EOcomb + n * device_gap_EOcomb)
    EOcomb_ref.dmovey(-EOcomb_ref.ports["o1"].dcenter[1] + EOcomb_ref.dysize + device_yoffset_EOcomb + (2 - n) * 5)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(EOcomb_ref.ports["o2"].dcenter[0] + EOcomb_ref.dxsize + routing_xoffset_eo_comb, EOcomb_ref.ports["o2"].dcenter[1]),
                   (EOcomb_ref.ports["o2"].dcenter[0] + EOcomb_ref.dxsize + routing_xoffset_eo_comb, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb + (n + 2) * 5),
                   (ecs["in"].ports["o2"].dcenter[0] + (n + 1) * 5, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb + (n + 2) * 5),
                   (ecs["in"].ports["o2"].dcenter[0]+ (n + 1) * 5, routing_yoffset_eo_comb2 - (n + 1) * 5),
                   (circuit7_saved_out_waypoint + (n + 1) * 5, routing_yoffset_eo_comb2 - (n + 1) * 5)] + circuit16_routing_list
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])

    return c

n = 2

x_in = 100 + (n + 7) * ports_gap + frame
x_out = circuit7_saved_out_waypoint_list[-1][0] + (n + 1) * ports_gap

in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1
}

circuit17_routing_list = [(x + (n + 1) * 5, y - (n + 1) * 5) for (x, y) in circuit7_saved_out_waypoint_list]
circuit17_routing_list[-1] = (ecs["out1"].ports["o2"].dcenter[0], circuit17_routing_list[-1][1])

@gf.cell
def EOcomb_circuit17(
    EOcomb = EOcomb,
    ecs = ecs,
    n = 2
) -> gf.Component:

    c = gf.Component()
    EOcomb_ref = c << EOcomb
    EOcomb_ref.dmovex(-EOcomb_ref.ports["o1"].dcenter[0] + n * EOcomb_ref.dxsize + device_xoffset_EOcomb + n * device_gap_EOcomb)
    EOcomb_ref.dmovey(-EOcomb_ref.ports["o1"].dcenter[1] + EOcomb_ref.dysize + device_yoffset_EOcomb + (2 - n) * 5)

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o1"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
    )

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o2"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(EOcomb_ref.ports["o2"].dcenter[0] + EOcomb_ref.dxsize + routing_xoffset_eo_comb, EOcomb_ref.ports["o2"].dcenter[1]),
                   (EOcomb_ref.ports["o2"].dcenter[0] + EOcomb_ref.dxsize + routing_xoffset_eo_comb, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb + (n + 3) * 5),
                   (ecs["in"].ports["o2"].dcenter[0] + (n + 1) * 5, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb + (n + 3) * 5),
                   (ecs["in"].ports["o2"].dcenter[0]+ (n + 1) * 5, routing_yoffset_eo_comb2 - (n + 1) * 5),
                   (circuit7_saved_out_waypoint + (n + 1) * 5, routing_yoffset_eo_comb2 - (n + 1) * 5)] + circuit17_routing_list
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])

    return c

########################
# TAU racetracks
########################

EOcomb = devices.EOcomb()

routing_xoffset_eo_comb_tau = -620 
outing_xoffset_eo_comb2_tau = 1000 
outing_xoffset_eo_comb3_tau = 5000
routing_yoffset_eo_comb_tau = 250
routing_yoffset_eo_comb2_tau = 1600
routing_yoffset_eo_comb3_tau = -270
device_gap_EOcomb_tau = 50

device_xoffset_EOcomb_tau = 1000.0
device_yoffset_EOcomb_tau = 860.0
in_port_xoffset_EOcomb_tau = 700
out_port_xoffset_EOcomb_tau = 1400

n = 0

x_in = 6000 + (n + 7) * ports_gap + frame 
x_out = circuit7_saved_out_waypoint_list[-1][0] + (n + 1) * 4 * ports_gap

in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1
}

circuit18_routing_list = [(x + (n + 4) * 5, y - (n + 4) * 5) for (x, y) in circuit7_saved_out_waypoint_list]
circuit18_routing_list[-1] = (ecs["out1"].ports["o2"].dcenter[0], circuit18_routing_list[-1][1])

@gf.cell
def EOcomb_tau_circuit18(
    EOcomb = EOcomb,
    ecs = ecs,
    n = 0
) -> gf.Component:

    c = gf.Component()
    EOcomb_ref = c << EOcomb
    EOcomb_ref.dmovex(-EOcomb_ref.ports["o1"].dcenter[0] + n * EOcomb_ref.dxsize + device_xoffset_EOcomb_tau)
    EOcomb_ref.dmovey(-EOcomb_ref.ports["o1"].dcenter[1] + EOcomb_ref.dysize + device_yoffset_EOcomb_tau + n * 5)

    global circuit18_saved_out_waypoint
    circuit18_saved_out_waypoint = ecs["in"].ports["o2"].dcenter[0]

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o2"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau, EOcomb_ref.ports["o2"].dcenter[1]),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb3_tau),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau + outing_xoffset_eo_comb3_tau  + n * 5, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb3_tau),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau + outing_xoffset_eo_comb3_tau  + n * 5, EOcomb_ref.ports["o2"].dcenter[1] + 2.3 * routing_yoffset_eo_comb3_tau - n * 5),
                   (ecs["in"].ports["o2"].dcenter[0], EOcomb_ref.ports["o2"].dcenter[1] + 2.3 * routing_yoffset_eo_comb3_tau - n * 5)]
        
    )

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o1"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[
                
                   (645 + (n + 4) * 5, EOcomb_ref.ports["o1"].dcenter[1]),
                   (645 + (n + 4) * 5, routing_yoffset_eo_comb2_tau - (n + 4) * 5),
                   (circuit7_saved_out_waypoint + (n + 4) * 5, routing_yoffset_eo_comb2_tau - (n + 4) * 5)] + circuit18_routing_list
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])

    return c

n = 1

x_in = 6000 + (n + 7) * ports_gap + frame 
x_out = circuit7_saved_out_waypoint_list[-1][0] + (n + 1) * 4 * ports_gap

in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1
}

circuit19_routing_list = [(x + (n + 1) * 5, y - (n + 1) * 5) for (x, y) in circuit7_saved_out_waypoint_list]
circuit19_routing_list[-1] = (ecs["out1"].ports["o2"].dcenter[0], circuit19_routing_list[-1][1])

@gf.cell
def EOcomb_tau_circuit19(
    EOcomb = EOcomb,
    ecs = ecs,
    n = 1
) -> gf.Component:

    c = gf.Component()
    EOcomb_ref = c << EOcomb
    EOcomb_ref.dmovex(-EOcomb_ref.ports["o1"].dcenter[0] + n * EOcomb_ref.dxsize + device_xoffset_EOcomb_tau)
    EOcomb_ref.dmovey(-EOcomb_ref.ports["o1"].dcenter[1] + EOcomb_ref.dysize + device_yoffset_EOcomb_tau + n * 5)

    global circuit19_saved_out_waypoint
    circuit19_saved_out_waypoint = ecs["in"].ports["o2"].dcenter[0]

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o2"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau, EOcomb_ref.ports["o2"].dcenter[1]),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb3_tau),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau + outing_xoffset_eo_comb3_tau  + n * 5, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb3_tau),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau + outing_xoffset_eo_comb3_tau  + n * 5, EOcomb_ref.ports["o2"].dcenter[1] + 2.3 * routing_yoffset_eo_comb3_tau - n * 5),
                   (ecs["in"].ports["o2"].dcenter[0], EOcomb_ref.ports["o2"].dcenter[1] + 2.3 * routing_yoffset_eo_comb3_tau - n * 5)]
        
    )

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o1"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[
                   (645 + (n + 4) * 5, EOcomb_ref.ports["o1"].dcenter[1]),
                   (645 + (n + 4) * 5, routing_yoffset_eo_comb2_tau - (n + 4) * 5),
                   (circuit7_saved_out_waypoint + (n + 4) * 5, routing_yoffset_eo_comb2_tau - (n + 4) * 5)] + circuit19_routing_list
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])

    return c

n = 2

x_in = 6000 + (n + 7) * ports_gap + frame 
x_out = circuit7_saved_out_waypoint_list[-1][0] + (n + 1) * 4 * ports_gap

in_loc = np.array([x_in, 0])
out_loc = np.array([x_out, chip_layout.dymax])

ec_in = gf.Component()
ec_ref = ec_in << double_taper
ec_ref.drotate(90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, in_loc - [0.0, 0.5 * input_ext]
)
ec_in.add_ports(ec_ref.ports)

ec_out1 = gf.Component()
ec_ref = ec_out1 << double_taper
ec_ref.drotate(-90.0)
ec_ref.dmove(
    ec_ref.ports["o1"].dcenter, out_loc + [0.0, 0.5 * input_ext]
)
ec_out1.add_ports(ec_ref.ports)

ecs = {
    "in": ec_in,
    "out1": ec_out1
}

circuit20_routing_list = [(x + (n + 1) * 5, y - (n + 1) * 5) for (x, y) in circuit7_saved_out_waypoint_list]
circuit20_routing_list[-1] = (ecs["out1"].ports["o2"].dcenter[0], circuit20_routing_list[-1][1])

@gf.cell
def EOcomb_tau_circuit20(
    EOcomb = EOcomb,
    ecs = ecs,
    n = 2
) -> gf.Component:

    c = gf.Component()
    EOcomb_ref = c << EOcomb
    EOcomb_ref.dmovex(-EOcomb_ref.ports["o1"].dcenter[0] + n * EOcomb_ref.dxsize + device_xoffset_EOcomb_tau)
    EOcomb_ref.dmovey(-EOcomb_ref.ports["o1"].dcenter[1] + EOcomb_ref.dysize + device_yoffset_EOcomb_tau + n * 5)

    global circuit20_saved_out_waypoint
    circuit20_saved_out_waypoint = ecs["in"].ports["o2"].dcenter[0]

    # Bend spec

    routing_bend = partial(
        gf.components.bend_euler,
        radius=routing_roc,
        with_arc_floorplan=True,
    )

    # Routing to I/O

    [c << ec for ec in ecs.values()]

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o2"],
        ecs["in"].ports["o2"],
        start_straight_length=0.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[(EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau, EOcomb_ref.ports["o2"].dcenter[1]),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb3_tau),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau + outing_xoffset_eo_comb3_tau  + n * 5, EOcomb_ref.ports["o2"].dcenter[1] + routing_yoffset_eo_comb3_tau),
                   (EOcomb_ref.ports["o2"].dcenter[0] + outing_xoffset_eo_comb2_tau + outing_xoffset_eo_comb3_tau  + n * 5, EOcomb_ref.ports["o2"].dcenter[1] + 2.3 * routing_yoffset_eo_comb3_tau - n * 5),
                   (ecs["in"].ports["o2"].dcenter[0], EOcomb_ref.ports["o2"].dcenter[1] + 2.3 * routing_yoffset_eo_comb3_tau - n * 5)]
        
    )

    gf.routing.route_single(
        c,
        EOcomb_ref.ports["o1"],
        ecs["out1"].ports["o2"],
        start_straight_length=5.0,
        end_straight_length=5.0,
        cross_section="xs_rwg1000",
        bend=routing_bend,
        straight="straight_rwg1000",
        waypoints=[
                
                   (645 + (n + 4) * 5, EOcomb_ref.ports["o1"].dcenter[1]),
                   (645 + (n + 4) * 5, routing_yoffset_eo_comb2_tau - (n + 4) * 5),
                   (circuit7_saved_out_waypoint + (n + 4) * 5, routing_yoffset_eo_comb2_tau - (n + 4) * 5)] + circuit20_routing_list
    )

    c.flatten()
    c.add_port(name="o1", port=ecs["in"].ports["o1"])
    c.add_port(name="o2", port=ecs["out1"].ports["o1"])

    return c


## dOR_EOM Zone
# dOR_EOM Short
# 30GHz
circuit2 = eom_ec_emmi_circuit2(eom_ec_emmi = devices.dOR_EOM_DC_EMMI(length_tapered = 300.0, ls = 1436.3, lextra = 355.0/2, DC_coupl_wg_sep = 0.570871))
circuit3 = eom_ec_emmi_circuit3(eom_ec_emmi = devices.dOR_EOM_DC_EMMI(length_tapered = 300.0, ls = 1436.3, lextra = 355.0/2, DC_coupl_wg_sep = 0.775075))
circuit4 = eom_ec_emmi_circuit4(eom_ec_emmi = devices.dOR_EOM_DC_EMMI(length_tapered = 300.0, ls = 1436.3, lextra = 355.0/2, DC_coupl_wg_sep = 0.889189))
circuit5 = eom_ec_emmi_circuit5(eom_ec_emmi = devices.dOR_EOM_DC_EMMI(length_tapered = 300.0, ls = 1436.3, lextra = 355.0/2, DC_coupl_wg_sep = 1.01772))

# dOR_EOM Long
# 15GHz
circuit6 = eom_ec_emmi_circuit6(eom_ec_emmi = devices.dOR_EOM_DC_EMMI(length_tapered = 300.0, ls = 4020.7, lextra = 37.0/2, DC_coupl_wg_sep = 0.812312))
circuit7 = eom_ec_emmi_circuit7(eom_ec_emmi = devices.dOR_EOM_DC_EMMI(length_tapered = 300.0, ls = 4020.7, lextra = 37.0/2, DC_coupl_wg_sep = 0.889189))
circuit8 = eom_ec_emmi_circuit8(eom_ec_emmi = devices.dOR_EOM_DC_EMMI(length_tapered = 300.0, ls = 4020.7, lextra = 37.0/2, DC_coupl_wg_sep = 0.967267))

## comb zone

respath = 'tofab_res.gds'#\"/Users/gab/PICLab Dropbox/personal/gsant/LXT_MPW/combs/microwave_design/tofab_res.gds\"\n",
padpath = 'tofab_pad_short.gds'#\"/Users/gab/PICLab Dropbox/personal/gsant/LXT_MPW/combs/microwave_design/tofab_pad_short.gds\"\n",
circuit9  =  duel_eo_comb_circuit9(duel_eo_comb=devices.dualEOcomb_sbend(
    ls = 1986.7,
    DC_coupl_wg_sep = 1.01772,
    RF_res_layout_path = respath,
    RF_pad_layout_path = padpath,
    ))
circuit10 = duel_eo_comb_circuit10(duel_eo_comb=devices.dualEOcomb_sbend(
    ls = 1882.1,
    DC_coupl_wg_sep = 1.01772,
    RF_res_layout_path = respath,
    RF_pad_layout_path = padpath,
    ))
circuit11 = duel_eo_comb_circuit11(duel_eo_comb=devices.dualEOcomb_sbend(
    ls = 1788.0,
    DC_coupl_wg_sep = 1.01772,
    RF_res_layout_path = respath,
    RF_pad_layout_path = padpath,
    ))
circuit12 = duel_eo_comb_circuit12(duel_eo_comb=devices.dualEOcomb_sbend(
    ls = 1702.9,
    DC_coupl_wg_sep = 1.01772,
    RF_res_layout_path = respath,
    RF_pad_layout_path = padpath,
    ))
circuit13 = duel_eo_comb_circuit13(duel_eo_comb=devices.dualEOcomb_sbend(
    ls = 1625.5,
    DC_coupl_wg_sep = 1.01772,
    RF_res_layout_path = respath,
    RF_pad_layout_path = padpath,
    ))
circuit14 = duel_eo_comb_circuit14(duel_eo_comb=devices.dualEOcomb_sbend(
    ls = 1554.8,
    DC_coupl_wg_sep = 1.01772,
    RF_res_layout_path = respath,
    RF_pad_layout_path = padpath,
    ))
circuit15 = EOcomb_circuit15(EOcomb = devices.EOcomb(
    RF_res_layout_path = 'TWGM_short_res_nm_flip.gds',
    RF_pad_layout_path = 'TWGM_short_pad_nm_flip.gds',
    RF_pad_pos = "bottom",
    RF_flipy = True,
    DC_coupl_wg_sep = 0.6,
    RF_pad_gap = 0,))
circuit16 = EOcomb_circuit16(EOcomb = devices.EOcomb(
    RF_res_layout_path = 'TWGM_short_res_nm_flip.gds',
    RF_pad_layout_path = 'TWGM_short_pad_nm_flip.gds',
    RF_pad_pos = "bottom",
    DC_coupl_wg_sep = 0.889,
    RF_flipy = True,
    RF_pad_gap = 0,))

circuit17 = EOcomb_circuit17(EOcomb = devices.EOcomb(
    RF_res_layout_path = 'TWGM_short_res_nm_flip.gds',
    RF_pad_layout_path = 'TWGM_short_pad_nm_flip.gds',
    RF_pad_pos = "bottom",
    DC_coupl_wg_sep = 1.01772,
    RF_pad_gap = 0,))

circuit18 = EOcomb_tau_circuit18(EOcomb = devices.EOcomb(
    DC_coupl_wg_sep = 1.01772,
    opt_flipy = True,
    ls =  1222.9))

circuit19 = EOcomb_tau_circuit18(EOcomb = devices.EOcomb(
    DC_coupl_wg_sep = 1.01772,
    opt_flipy = True,
    ls =  1222.9))

circuit20 = EOcomb_tau_circuit20(EOcomb = devices.EOcomb(
    DC_coupl_wg_sep = 1.01772,
    opt_flipy = True,
    ls =  1222.9))

@gf.cell
def die_assembled(
    chip_layout = chip_layout,
    circuit = circuit,
    circuit2 = circuit2,
    circuit3 = circuit3,
    circuit4 = circuit4,
    circuit5 = circuit5,
    circuit6 = circuit6,
    circuit7 = circuit7,
    circuit8 = circuit8,
    circuit9 = circuit9,
    circuit10 = circuit10,
    circuit11 = circuit11,
    circuit12 = circuit12,
    circuit13 = circuit13,
    circuit14 = circuit14,
    circuit15 = circuit15,
    circuit16 = circuit16,
    circuit17 = circuit17,
    circuit18 = circuit18,
    circuit19 = circuit19,
    circuit20 = circuit20

) -> gf.Component:
    c = gf.Component()
    c << chip_layout
    c << circuit
    c << circuit2
    c << circuit3
    c << circuit4
    c << circuit5
    c << circuit6
    c << circuit7
    """ c << circuit8 """
    c << circuit9
    c << circuit10
    c << circuit11
    c << circuit12
    c << circuit13
    """ c << circuit14 """
    c << circuit15
    c << circuit16
    c << circuit17
    c << circuit18
    c << circuit19
    c << circuit20

    """ c.add_ports(circuit.ports)
    c.add_ports(circuit2.ports) """
    return c

die = die_assembled()
die.plot()
die.show()
""" _ = die.write_gds(gdsdir=Path.cwd()) """



(598.953, 17.0)


PlacerError: Error while trying to place route from o1 to o2 at points (dbu): [4181612,1107009, 675000,1107009, 675000,1570000, 5548950,1570000, 5533950,2869310, 5961282,2869310, 5961282,4181130, 8013465,4181130, 8013465,4685000]

In [11]:
5560950- 5521250

39700

In [12]:
die.pprint_ports()

┏━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┓
┃ name ┃ width ┃ orientation ┃ layer ┃ center ┃ port_type ┃
┡━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━┩
└──────┴───────┴─────────────┴───────┴────────┴───────────┘

In [13]:
gf.clear_cache()